In [1]:
from src.problems.domino import Domino, bigger, random_player, hardest, random_players, random_hands, hardest_bigger
from src.choices.logChoice import LogChoice
from src.choices.bigChoice import BigChoice
from src.choicesParameters.fixedParameters import FixedParameters
import numpy as np
import random
from src.choicesParameters.metaheuristicParameters import PSOParameters
from src.choicesParameters.transformerParameters import TransformerParametersSVD, TransformerParametersSequential

In [2]:
choice = LogChoice()

choiceParameters = FixedParameters()
problem = Domino(choice, choiceParameters)

psoParameters = PSOParameters()
problemFit = Domino(choice, psoParameters)

In [3]:
def get_sequences_from_start(list):
    return [list[:pos + 1] for pos in range(len(list))]

In [4]:
X, Y = [], []

In [ ]:
for c in range(500):
    players, main_player = random_players([bigger, hardest, random_player], 4, hardest_bigger)
    hands = random_hands(len(players))

    count_fitness = 100

    def fitness(p):
        losses = 0

        choiceParameters.set(p[0])

        for i in range(count_fitness):
            ch = [[f for f in hand] for hand in hands]
            ws = problem.run(ch, players, main_player)

            if isinstance(ws, int) and ws != main_player:
                losses += 1
            elif isinstance(ws, list) and main_player not in ws:
                losses += 1

        return losses

    score, p = psoParameters.fit(fitness, choice.apply_constraints, choice.count_params(), 20, 25)
    print(score)

    ch = [[f for f in hand] for hand in hands]
    problemFit.run(ch, players, main_player)

    X.extend(get_sequences_from_start(problemFit.embeddings))
    Y.extend([p] * len(problemFit.embeddings))

In [6]:
transformerParameters = TransformerParametersSVD(2, 256)

In [ ]:
preds, error = transformerParameters.train(X, Y, 100)
print(error)

In [ ]:
problemTrans = Domino(choice, transformerParameters)

count = 200
losses = 0

for i in range(count):
    players, main_player = random_players([bigger, hardest, random_player], 4, hardest_bigger)
    hands = random_hands(len(players))
    ws = problemTrans.run(hands, players, main_player)

    if isinstance(ws, int) and ws != main_player:
        losses += 1
    elif isinstance(ws, list) and main_player not in ws:
        losses += 1

print(losses)